### IMPORTANDO BIBLIOTECAS

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

from math import sqrt

  0%|          | 0.00/578k [00:13<?, ?B/s]




A biblioteca `gdown` foi utilizada para facilitar o download de arquivos diretamente do Google Drive. Essa abordagem é especialmente útil quando se trabalha em ambientes colaborativos, como o Google Colab, onde o acesso a arquivos compartilhados é necessário para a continuidade do trabalho.

#### Passos Realizados com gdown:

1. **Importação da Biblioteca**:
    A biblioteca `gdown` foi importada para permitir o download de arquivos a partir de URLs do Google Drive.

2. **Definição do ID do Arquivo**:
    O ID do arquivo desejado foi especificado, permitindo a construção da URL de download.

3. **Download do Arquivo**:
    Utilizando a função `gdown.download()`, o arquivo foi baixado e salvo localmente com o nome `dataframe_alterado.csv`.

Essa abordagem simplifica o processo de obtenção de dados, garantindo que todos os colaboradores tenham acesso aos mesmos arquivos de forma rápida e eficiente.


In [ ]:
# baixando o gdown para baixar o arquivo

!pip install gdown

In [ ]:
import gdown

# Colocar o ID do arquivo aqui (substitua 'FILE_ID' pelo ID correto)
file_id = '1BeB7U-Y-wN5L5XnSw-QIpph4MXd93g0V'
url = f'https://drive.google.com/uc?id={file_id}'

# - https://drive.google.com/file/d/1BeB7U-Y-wN5L5XnSw-QIpph4MXd93g0V/view?usp=sharing

# Baixar o arquivo
gdown.download(url, 'dataframe_alterado.csv',quiet=False)

In [12]:
df = pd.read_csv('dataframe_alterado.csv')

# Modelagem e Features utilizadas

&nbsp;&nbsp;&nbsp;&nbsp;Para desenvolver o modelo preditivo, selecionamos as features mais relevantes para análise e treinamento, priorizando aquelas que impactam diretamente as receitas futuras da Rede Gazeta. A escolha dessas variáveis visa garantir um modelo preciso e alinhado com as necessidades estratégicas da empresa.

**Ano:** Captura tendências de longo prazo nas receitas, refletindo impactos de mudanças estratégicas e condições econômicas que afetam o desempenho anual.

**Mês:** Considera variações sazonais, permitindo uma análise detalhada de flutuações mensais nas receitas, ajustando previsões de acordo com períodos de alta e baixa demanda.

**UEN (Unidade Estratégica de Negócio):** Distingue as receitas por setor (TV, rádio, jornal, digital), facilitando a identificação de oportunidades e ajustes estratégicos para maximizar o potencial de cada unidade.

**Veículo:** Reflete o impacto de diferentes canais de comunicação nas receitas, auxiliando na otimização das estratégias de marketing e alocação de recursos.

**Valor Líquido Final:** Representa o valor final dos contratos após descontos, categorizado para capturar a realidade financeira da empresa, crucial para previsões mais realistas.

&nbsp;&nbsp;&nbsp;&nbsp;Essas features, combinadas, proporcionam uma visão abrangente e detalhada das operações financeiras da Rede Gazeta, permitindo um modelo preditivo robusto que otimiza a alocação de recursos, melhora a precisão das previsões e apoia decisões estratégicas informadas.

In [ ]:
df.head()

In [18]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder() # Instanciando o LabelEncoder


df['UEN'] = label_encoder.fit_transform(df['UEN'])
df['Segmento'] = label_encoder.fit_transform(df['Segmento'])
df['Setor'] = label_encoder.fit_transform(df['Setor'])
df['Veiculo'] = label_encoder.fit_transform(df['Veiculo'])
df['Origem'] = label_encoder.fit_transform(df['Origem'])

In [ ]:
# Convertendo a coluna "Mês/ano" para datetime
df['Mês/ano'] = pd.to_datetime(df['Mês/ano'])

df['Ano'] = df['Mês/ano'].dt.year
df['Mês'] = df['Mês/ano'].dt.month
df.set_index(['Ano', 'Mês'], inplace=True)

# Visualizando o resultado
df.head()

In [ ]:
df.info()

In [ ]:
# Agrupar por 'Ano', 'Mês', 'UEN', e 'Veiculo', somando os valores de 'VL Liquido Final'
df_grouped = df.groupby(['Ano', 'Mês', 'UEN', 'Veiculo'])['Vl Liquido Final'].sum().reset_index()

# Exibir as primeiras linhas do DataFrame agrupado
df_grouped.head(10)


In [ ]:
df = df_grouped

## Intervalos

In [ ]:
# Criar os intervalos de 50
bins = range(0, int(df['Vl Liquido Final'].max()) + 50, 50)

# Transformar a coluna 'Vl Liquido Final' em categorias de intervalos
df['Vl Liquido Final Categorias'] = pd.cut(df['Vl Liquido Final'], bins=bins, labels=bins[:-1])

# - excluindo a coluna 'Vl liquido Final'

# df = df.drop(['Vl Liquido Final'], axis=1)

# Exibir o resultado
display(df.head(10))



In [ ]:
X

In [ ]:
y

In [ ]:
# Definindo X e y a partir do DataFrame
X = df[['Ano', 'Mês', 'UEN', 'Veiculo']]  # Variáveis preditoras (já encodadas)
y = df['Vl Liquido Final']  # Variável alvo (valores a serem previstos)

# Dividindo os dados em treino e teste (80% treino, 20% teste)
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)

# Criando o modelo de Random Forest para regressão
modelo = RandomForestRegressor(n_estimators=100, random_state=42)

# Treinando o modelo
modelo.fit(X_treino, y_treino)

# Fazendo previsões nos dados de teste
previsoes = modelo.predict(X_teste)

# Avaliando o modelo
r2_score = modelo.score(X_teste, y_teste)  # Coeficiente de determinação R²
mae = mean_absolute_error(y_teste, previsoes)  # Erro absoluto médio
mse = mean_squared_error(y_teste, previsoes)  # Erro quadrático médio
mape = mean_absolute_percentage_error(y_teste, previsoes) * 100

# Exibindo as métricas de avaliação
print(f"R²: {r2_score}")
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f'RMSE: {np.sqrt(mse)}')
print(f'MAPE: {(mape)}')
print(f'Predições: {previsoes}')
print(f'Valores de y:{np.array(y_teste)}')

In [ ]:
df_branco = pd.read_csv('/content/drive/Shareddrives/Hipóteses/DadosEconomicos_ES_Inteli.xlsx - VarejoTotal.csv')

# Tuning de hiperparâmetro

In [ ]:
# Função para coletar os dados via input do usuário
def coletar_dados_usuario():
    # Solicitando ao usuário os inputs
    ano = int(input("Digite o ano: "))
    mes = int(input("Digite o mês: "))
    uen = int(input("Digite a UEN (numero referente): "))
    veiculo = int(input("Digite o veículo (numero referente): "))

    # Criando um DataFrame com os dados fornecidos
    dados_usuario = pd.DataFrame({
        'Ano': [ano],
        'Mês': [mes],
        'UEN': [uen],
        'Veiculo': [veiculo]
    })

    return dados_usuario

# Coletando os dados do usuário
novos_dados = coletar_dados_usuario()

# Fazendo a previsão com o modelo treinado
previsao = modelo.predict(novos_dados)

# Exibindo a previsão
print(f"A renda prevista nesses parametros é de: R${previsao[0]:.2f}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score

# Verificando se o DataFrame 'df' está carregado corretamente
try:
    print("Visualizando as primeiras linhas do DataFrame:")
    print(df.head())
except NameError:
    print("Erro: DataFrame 'df' não está definido. Verifique se foi carregado corretamente.")
    # Caso o df não esteja carregado, carregue os dados aqui

# Garantir que a coluna 'Vl Liquido Final' existe
if 'Vl Liquido Final' in df.columns:
    # Criar os intervalos de 50 unidades para categorização
    bins = range(0, int(df['Vl Liquido Final'].max()) + 50, 50)
    df['Vl Liquido Final Categorias'] = pd.cut(df['Vl Liquido Final'], bins=bins, labels=bins[:-1])
else:
    print("Erro: Coluna 'Vl Liquido Final' não encontrada no DataFrame.")

# Definindo X e y
X = df[['Ano', 'Mês', 'UEN', 'Veiculo']]  # Variáveis preditoras
y = df['Vl Liquido Final Categorias']  # Variável alvo categorizada

# Dividindo os dados em treino e teste (80% treino, 20% teste)
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir o modelo base
modelo = RandomForestClassifier(random_state=42)

# Definindo o grid de hiperparâmetros que serão testados
param_grid = {
    'n_estimators': [50, 100, 200],            # Número de árvores
    'max_depth': [None, 10, 20, 30],           # Profundidade máxima
    'min_samples_split': [2, 5, 10],           # Mínimo de amostras para split
    'min_samples_leaf': [1, 2, 4],             # Mínimo de amostras por folha
    'bootstrap': [True, False]                 # Usar ou não amostragem com reposição
}

# Instanciar o GridSearchCV
grid_search = GridSearchCV(estimator=modelo, param_grid=param_grid,
                           cv=3, n_jobs=-1, verbose=2)

# Realizando a busca pelos melhores hiperparâmetros
print("Iniciando GridSearchCV...")
grid_search.fit(X_treino, y_treino)

# Exibir os melhores hiperparâmetros
print(f"Melhores hiperparâmetros encontrados: {grid_search.best_params_}")

# Treinar o modelo final com os melhores parâmetros
best_model = grid_search.best_estimator_

# Fazer previsões nos dados de teste
previsoes = best_model.predict(X_teste)

# Avaliar o modelo
accuracy = accuracy_score(y_teste, previsoes)
print(f"Acurácia: {accuracy}")
print("Relatório de Classificação:")
print(classification_report(y_teste, previsoes))

# Exibir as previsões e os valores reais
print(f"Previsões: {previsoes[:10]}")
print(f"Valores reais: {y_teste[:10].values}")


# Dicionário
### Dicionário de categorias

#### UEN:
- DIGITAL: 0
- Rádio: 1
- Televisão: 2

#### Veiculo:
- CBN VITORIA: 0
- GAZETA FM LINHARES: 1
- GAZETA FM VITÓRIA: 2
- GAZETA PRODUCOES: 3
- INTERNET: 4
- LITORAL FM: 5
- LITORAL FM NOROESTE: 6
- LITORAL FM NORTE: 7
- LITORAL FM SUL: 8
- PORTAL G1/GE/HOME: 9
- PRODUCAO: 10
- RÁDIO MIX VITÓRIA: 11
- TV GAZETA: 12
- TV GAZETA NOROESTE: 13
- TV GAZETA NORTE: 14
- TV GAZETA SUL: 15
- TV NOROESTE PRODUCOES: 16
- TV NORTE PRODUCOES: 17
- TV SUL PRODUCOES: 18

# Por que Random Forest?

&nbsp;&nbsp;&nbsp;&nbsp;Random Forest é um algoritmo de aprendizado de máquina baseado em árvores de decisão, usado para tarefas de classificação e regressão. Esse algoritmo funciona criando múltiplas árvores de decisão independentes, onde cada árvore é treinada com diferentes amostras dos dados e subconjuntos de features.

&nbsp;&nbsp;&nbsp;&nbsp;O objetivo principal do Random Forest é combinar as previsões de todas as árvores para obter um resultado mais robusto. No caso de classificação, a previsão final é feita com base na maioria das votações das árvores; para regressão, a média das previsões é utilizada.

&nbsp;&nbsp;&nbsp;&nbsp;No contexto do projeto, o  Random Forest é uma escolha eficiente para o modelo de previsão de receita da Rede Gazeta, considerando as características dos dados e os objetivos do projeto. O desafio principal é prever o VL Líquido Final com base em variáveis como "Ano", "Mês", "UEN" e "Veículo". Cada uma dessas variáveis traz suas próprias particularidades, tornando necessária a escolha de um modelo que consiga lidar com a variabilidade dos dados, as relações não lineares e as interações entre elas.

&nbsp;&nbsp;&nbsp;&nbsp;Os dados fornecidos possuem uma característica multidimensional. Variáveis como UEN e Veículo podem estar associadas a diferentes unidades de negócios da Rede Gazeta, como TV, rádio e internet, cada uma apresentando padrões de receita distintos ao longo do tempo. O Random Forest se destaca na análise dessas múltiplas dimensões, pois ao trabalhar com combinações aleatórias de subconjuntos de variáveis em suas árvores, ele é capaz de capturar interações sutis e padrões que não são óbvios à primeira vista. Além disso, as variáveis temporais, como Ano e Mês, introduzem uma dimensão temporal importante, e o modelo, por sua natureza, consegue identificar automaticamente tendências sazonais e padrões temporais sem a necessidade de transformações complexas, o que facilita a análise de flutuações de receita em períodos específicos.

&nbsp;&nbsp;&nbsp;&nbsp;Outro ponto importante é a capacidade do Random Forest de lidar com interações não lineares entre as variáveis e a receita final. Pode haver, por exemplo, uma relação complexa entre UEN e Veículo, onde determinados veículos de mídia performam melhor para certas unidades de negócios em períodos específicos do ano. O Random Forest se destaca nesse aspecto, já que suas árvores de decisão exploram diferentes divisões e interações entre as variáveis, permitindo que o modelo capte essas relações sem a necessidade de assumir uma linearidade entre os dados, como acontece com modelos mais tradicionais, como a regressão linear.

&nbsp;&nbsp;&nbsp;&nbsp;Além disso, o Random Forest se mostra resistente a outliers e ruídos nos dados, o que é especialmente importante em previsões financeiras. Muitas vezes, eventos atípicos, promoções pontuais ou campanhas específicas podem gerar picos temporários na receita, e o modelo consegue lidar bem com esses desvios, tornando-o uma ferramenta mais estável e confiável quando comparado a outros modelos, como redes neurais ou regressões lineares, que podem ser mais sensíveis a essas variações nos dados.

&nbsp;&nbsp;&nbsp;&nbsp;No mais, um dos maiores desafios em qualquer projeto preditivo é evitar o overfitting, que ocorre quando o modelo se ajusta demais aos dados de treinamento e perde a capacidade de generalizar para novos dados. O Random Forest mitiga esse risco ao combinar os resultados de várias árvores de decisão, cada uma treinada com diferentes amostras dos dados. Dessa forma, o modelo resultante é menos propenso a se ajustar de maneira excessiva a ruídos ou peculiaridades dos dados de treino. Essa característica é importante em cenários onde as receitas podem ser impactadas por fatores externos imprevisíveis ou flutuações de mercado.

&nbsp;&nbsp;&nbsp;&nbsp;Outro aspecto que reforça a escolha do Random Forest é sua capacidade de identificar a importância de cada variável no processo de previsão. No projeto da Rede Gazeta, entender como variáveis como UEN ou Veículo influenciam o VL Líquido Final pode gerar insights valiosos para as estratégias de negócios da empresa. O modelo atribui uma importância relativa a cada variável, medindo sua contribuição para a redução do erro de previsão. Esse tipo de informação pode auxiliar a equipe da Rede Gazeta a identificar quais unidades de negócios ou veículos de mídia estão mais associados ao desempenho financeiro, permitindo ajustes mais precisos nas estratégias de marketing e alocação de recursos.

&nbsp;&nbsp;&nbsp;&nbsp;Por fim, a escalabilidade do Random Forest é outro fator decisivo, visto que o projeto envolve um grande volume de dados, que tende a crescer com o tempo, à medida que novas informações são coletadas. O Random Forest é capaz de lidar com essa expansão sem perda de desempenho, o que o torna uma ferramenta adequada para projetos de longo prazo.

&nbsp;&nbsp;&nbsp;&nbsp;Em resumo, a utilização do Random Forest no projeto de previsão de receitas da Rede Gazeta é amplamente justificada pela sua capacidade de lidar com variáveis complexas e multidimensionais, interações não lineares e outliers, além de oferecer resistência contra overfitting e escalabilidade para grandes volumes de dados. Sua flexibilidade e capacidade de fornecer insights práticos sobre a importância das variáveis envolvidas tornam esse modelo uma escolha ideal para projetos com essas características. Assim, o Random Forest não só contribui para previsões mais precisas, mas também oferece uma base sólida para decisões estratégicas no âmbito comercial da Rede Gazeta.

## Discussão dos resultados do modelo

&nbsp;&nbsp;&nbsp;&nbsp;Os resultados obtidos com o modelo de Random Forest demonstram uma performance forte e adequada ao contexto do projeto de previsão de receita da Rede Gazeta. Com um coeficiente de determinação (R²) de 0.967, o modelo mostra-se capaz de explicar aproximadamente 97% da variabilidade presente nos dados, o que indica uma importante correlação entre as variáveis preditoras e o VL Líquido Final. Esse alto valor de R² é indicativo de que o modelo consegue captar de forma eficaz os padrões presentes nas interações complexas entre variáveis como "Ano", "Mês", "UEN" e "Veículo".

&nbsp;&nbsp;&nbsp;&nbsp;O erro absoluto médio (MAE) de aproximadamente 61 mil sugere que, em média, o modelo erra por esse valor nas suas previsões. Embora possa parecer um erro considerável, ele deve ser avaliado em comparação com as magnitudes das receitas envolvidas. Vale destacar que o MAE é uma métrica que não amplifica grandes desvios, fornecendo uma visão direta da magnitude dos erros de previsão.

&nbsp;&nbsp;&nbsp;&nbsp;Já o erro quadrático médio (MSE) de 20 bilhões indica uma maior penalização para erros grandes, o que é esperado, dado que o MSE eleva as diferenças ao quadrado. Este valor, embora alto, deve ser analisado com cuidado, considerando que grandes variações em receitas podem ocorrer devido a eventos atípicos ou campanhas específicas, como mencionado anteriormente. O MSE, nesse caso, pode estar refletindo alguns outliers que o modelo, embora forte, captou.

&nbsp;&nbsp;&nbsp;&nbsp;Esses resultados reforçam a eficácia do Random Forest em lidar com a complexidade e variabilidade dos dados, além de sua capacidade de realizar previsões precisas sem a necessidade de transformações complexas. A alta performance do modelo também corrobora sua escolha inicial, especialmente por sua capacidade de reduzir o overfitting e identificar a importância das variáveis preditivas, fornecendo insights práticos que podem auxiliar nas estratégias de negócios da Rede Gazeta.


# Treino com ajustes dos Hiperparâmetros

In [ ]:
# Definindo X e y a partir do DataFrame
X = df[['Ano', 'Mês', 'UEN', 'Veiculo']]  # Variáveis preditoras (já encodadas)
y = df['Vl Liquido Final']  # Variável alvo (valores a serem previstos)

# Dividindo os dados em treino e teste (80% treino, 20% teste)
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)

# Criando o modelo de Random Forest para regressão
modelo = RandomForestRegressor(bootstrap= False, max_depth= None, min_samples_leaf= 1, min_samples_split= 10, n_estimators= 100)

# Treinando o modelo
modelo.fit(X_treino, y_treino)

# Fazendo previsões nos dados de teste
previsoes = modelo.predict(X_teste)

# Avaliando o modelo
r2_score = modelo.score(X_teste, y_teste)  # Coeficiente de determinação R²
mae = mean_absolute_error(y_teste, previsoes)  # Erro absoluto médio
mse = mean_squared_error(y_teste, previsoes)  # Erro quadrático médio
mape = mean_absolute_percentage_error(y_teste, previsoes) * 100

# Exibindo as métricas de avaliação
print(f"R²: {r2_score}")
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f'RMSE: {np.sqrt(mse)}')
print(f'MAPE: {(mape)}')
print(f'Predições: {previsoes}')
print(f'Valores de y:{np.array(y_teste)}')

# Modelos com Parâmetros Aleatórios vs. GridSearchCV

## Resultados com Parâmetros Aleatórios:

R²: 0.967

MAE: 61,324

MSE: 20,053,779,830.90

RMSE: 141,611.37

MAPE: 62.45%

## Resultados com GridSearchCV:

R²: 0.970

MAE: 64,698

MSE: 18,083,354,635.59

RMSE: 134,474.36

MAPE: 52.30%

## Análise Comparativa:

## Coeficiente de Determinação (R²):

O R² do modelo com parâmetros aleatórios é 0.967, enquanto o R² do modelo otimizado com GridSearchCV é 0.970. Isso mostra uma ligeira melhora na capacidade explicativa do modelo otimizado, indicando que os hiperparâmetros ajustados pelo GridSearchCV permitem que o modelo explique um pouco mais da variação nos dados.

## MAE (Erro Absoluto Médio):

O modelo otimizado tem um MAE ligeiramente maior (64,698) do que o modelo aleatório (61,324). Embora o MAE seja um pouco maior, isso pode ocorrer devido ao balanceamento de outros erros em classes diferentes. Essa métrica isolada não deve ser usada para descartar a otimização.

## MSE (Erro Quadrático Médio):

O MSE do modelo otimizado com GridSearchCV é menor (18,083,354,635.59) em comparação com o modelo com parâmetros aleatórios (20,053,779,830.90), indicando que o erro geral do modelo foi reduzido. Isso significa que o modelo otimizado comete erros menores, em média, quando faz previsões.

## RMSE (Raiz do Erro Quadrático Médio):

O RMSE do modelo otimizado é menor (134,474.36) em relação ao modelo aleatório (141,611.37), o que reforça que o modelo otimizado tem uma precisão melhor, pois o RMSE pondera erros grandes mais fortemente do que o MAE.

## MAPE (Erro Percentual Absoluto Médio):

O MAPE caiu significativamente no modelo otimizado (52.30% vs. 62.45%). Isso mostra que, em termos percentuais, o erro médio nas previsões em relação aos valores reais foi reduzido.

## Importância do GridSearchCV:
O processo de otimização de hiperparâmetros por meio do GridSearchCV é importante para melhorar a performance do modelo, porque ele testa várias combinações de parâmetros e encontra a configuração que proporciona os melhores resultados com base em uma métrica de avaliação. No contexto do projeto, utilizar um modelo otimizado permite:

Previsões Mais Precisas: Com um modelo otimizado, as previsões sobre o "Valor Líquido Final" podem ser mais acuradas, o que é útil para garantir justas e baseadas em dados confiáveis.

Balanceamento de Erros: O GridSearchCV equilibra a precisão do modelo ao otimizar aspectos, como a profundidade das árvores, o número de divisões e folhas, e o uso de amostragem com ou sem reposição, o que é necessário em um ambiente onde diferentes variáveis afetam a performance.

Conclusão:
Mesmo que o MAE tenha aumentado ligeiramente no modelo otimizado, outras métricas, como o MSE, RMSE, e especialmente o MAPE, mostram que o modelo melhorou. Esse ajuste é importante para garantir que a previsão seja segura.






